In [1]:
%run tools.py
%run analyze.py

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Import data

In [6]:
data_temperature = np.genfromtxt(fname = "data/real-world/4- for_compositional_analysis/temperature.csv", 
                                 delimiter = ',',
                                 skip_header = 1,
                                 dtype = None)

data_rain = np.genfromtxt(fname = "data/real-world/4- for_compositional_analysis/rain.csv", 
                                 delimiter = ',',
                                 skip_header = 1,
                                 dtype = None)

data_sales = np.genfromtxt(fname = "data/real-world/4- for_compositional_analysis/sales.csv", 
                                 delimiter = ',',
                                 skip_header = 1,
                                 dtype = None)

data_salary = np.genfromtxt(fname = "data/real-world/4- for_compositional_analysis/salary.csv", 
                                 delimiter = ',',
                                 skip_header = 1,
                                 dtype = None)

data_gym = np.genfromtxt(fname = "data/real-world/4- for_compositional_analysis/gym.csv", 
                                 delimiter = ',',
                                 skip_header = 1,
                                 dtype = None)

data_facebook = np.genfromtxt(fname = "data/real-world/4- for_compositional_analysis/facebook.csv", 
                                 delimiter = ',',
                                 skip_header = 1,
                                 dtype = None)

# Catmull-Rom

In [7]:
def data_to_points ( data ):
    #Transform the data to points
    points = []
    for i in data:
        x = i[7]
        y = i[2]

        points.append([x, y])
        
    return points

def do_catmull ( points ):
    return catmull_rom_chain(points, nPointsCatmullRom = 1000)

def reduce ( full_spline ):
    
    return full_spline

def divide_to_arrays ( matrix ):
    a0 = []
    a1 = []
    
    for i in matrix:
        a0.append([float(i[0])])
        a1.append([float(i[1])])
        
    return [np.array(a0), np.array(a1)]

def spline ( data ):
    points = data_to_points(data)
    
    full_spline = do_catmull(points)

    spline = reduce_spline(full_spline)
    
    return divide_to_arrays(spline)

In [8]:
spline_temperature = spline(data_temperature)
spline_rain = spline(data_rain)
spline_sales = spline(data_sales)
spline_salary = spline(data_salary)
spline_gym = spline(data_gym)
spline_facebook = spline(data_facebook)

# Gaussian Processes

In [9]:
def compute_gps_2(spline, name):
    print('START ' + name)
    
    #Subsample every fifth element
    X = spline[0][1::5]
    Y = spline[1][1::5]

    gps = compute_gps(X, Y)
    likelihoods = compute_lmls(gps)

    return {'gps': gps, 'lmls': likelihoods}

In [10]:
gps_temperature = compute_gps_2(spline_temperature, 'temperature')
gps_rain        = compute_gps_2(spline_rain,        'rain')
gps_sales       = compute_gps_2(spline_sales,       'sales')
gps_salary      = compute_gps_2(spline_salary,      'salary')
gps_gym         = compute_gps_2(spline_gym,         'gym')
gps_facebook    = compute_gps_2(spline_facebook,    'facebook')

START temperature
Adding White Kernel to l+r
Adding White Kernel to l*r*p
START rain
Adding White Kernel to l*r
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
START sales
Adding White Kernel to l*r
Adding White Kernel to l*r+p
Adding White Kernel to l*p+r
Exception caught computing l*p+r
Adding White Kernel to l*r*p
START salary
Adding White Kernel to l+r
Adding White Kernel to l+p
Exception caught computing l+p
Adding White Kernel to p+r
Adding White Kernel to l*r
Exception caught computing l*r
Adding White Kernel to l*p
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
Exception caught computing l*r*p
START gym
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l*r*p
START facebook
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to p*r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
Exception caught computing l*r*p


In [13]:
def get_max_kernel(gps, name):
    max_value = -1000000000
    max_kernel = ''

    for kernel in gps['lmls']:
        if max_value < gps['lmls'][kernel]['lml']:
            max_value = gps['lmls'][kernel]['lml']
            max_kernel = kernel
            
    print(name + ', ' + max_kernel + ', ' + str(max_value))

In [14]:
get_max_kernel(gps_temperature, 'temperature')
get_max_kernel(gps_rain, 'rain')
get_max_kernel(gps_sales, 'sales')
get_max_kernel(gps_salary, 'salary')
get_max_kernel(gps_gym, 'gym')
get_max_kernel(gps_facebook, 'facebook')

temperature, p*r, 423.889049111
rain, l+r*p, -188.085524333
sales, p+r, -53.8087448088
salary, r, -885.321235891
gym, l+r*p, 104.173505782
facebook, l+r+p, 1911.71253293


## Save results

In [15]:
import csv

In [16]:
def write_in_csv(writer, gps, name):
    for i in gps_temperature['lmls']:
        writer.writerow([name, i, gps['lmls'][i]['lml']])

In [17]:
with open('data/real-world/to-plot.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    
    writer.writerow(['scenario', 'kernel', 'lml'])
    
    write_in_csv(writer, gps_temperature, 'Temperature')
    write_in_csv(writer, gps_rain,        'Rain')
    write_in_csv(writer, gps_sales,       'Sales')
    write_in_csv(writer, gps_salary,      'Salary')
    write_in_csv(writer, gps_gym,         'Gym members')
    write_in_csv(writer, gps_facebook,    'FB Friends')

# Save splines
#### They will be used in the plots

In [36]:
def write_spline_in_csv(writer, spline, name):
    x = 0
    
    for y in spline[1]:
        writer.writerow([name, x, y[0]])
        x += 1

In [37]:
with open('data/real-world/splines.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    
    writer.writerow(['scenario', 'day', 'y'])
    
    write_spline_in_csv(writer, spline_temperature, 'Temperature')
    write_spline_in_csv(writer, spline_rain,        'Rain')
    write_spline_in_csv(writer, spline_sales,       'Sales')
    write_spline_in_csv(writer, spline_salary,      'Salary')
    write_spline_in_csv(writer, spline_gym,         'Gym members')
    write_spline_in_csv(writer, spline_facebook,    'FB Friends')